In [1]:
import pandas as pd

import plotly.express as px

from main import GameResult

In [2]:
n_fields = 50
animals_start_idx = 5
n_snacks = 4
print("Available game states:")
print(
    # CAT on n_fields
    n_fields \
    
    # CAT consumed n_snacks 
    * n_snacks \
    
    # MICE, SQUIRREL, BIRD on n_fields - animals_start_idx    |   **3 because of need to multiply for every animal
    * (n_fields - animals_start_idx) ** 3  \

    # every of MICE, SQUIRREL, BIRD could be IN_PROGRESS, CHASED, FINISHED   |   **3 because of need to multiply for every animal
    * 3 ** 3
)

Available game states:
492075000


In [17]:
def _sanity_check(x, CAT, animal_cols):
    cat_finished = x[CAT] == GameResult.FINISHED.name
    any_animal_finished = any(x[col] == GameResult.FINISHED.name for col in animal_cols)
    all_animals_chased = all(x[col] == GameResult.CHASED.name for col in animal_cols)
    if cat_finished and any_animal_finished:
        pass  # fine, for ex. animal can finish first and cat can finish afterwards (while other animals can finish later or could be chased beforehand by cat)
    if cat_finished and not all_animals_chased:
        pass # fine, for ex. cat can "skip" animal field in the meantime and be faster than other animals so that he reaches finish faster
    if any_animal_finished == all_animals_chased:
        raise Exception("Impossible, if any animal finished, at least one animal is not chased")


def get_winner(x):
    CAT = "C"
    animal_cols = ["B", "S", "M"]
    _sanity_check(x, CAT, animal_cols)
    result = None
    for col in [CAT] + animal_cols:
        if x[col] == GameResult.FINISHED.name:
            if result is None:
                result = col
            else:
                result += f"_{col}"
                # raise ValueError(f"W WTF? {result=}, {x=}")
    return result

In [4]:
df = pd.read_csv("results.csv")
display(df.describe())
display(df.groupby(by="strategy").describe())

,epoch,position,game_result,shortcut_position,snacks_cnt,game_id,max_num_snacks,timestamp
count,2.577636e+06,2.577636e+06,2.577636e+06,1.933227e+06,644409.000000,2.577636e+06,2.577636e+06,2.577636e+06
mean,3.551508e+01,1.902918e+01,-3.966390e-01,2.100000e+01,-0.603007,5.956723e+03,3.209968e+00,1.748333e+12
std,2.634428e+01,1.313080e+01,6.133651e+00,8.164968e+00,3.856585,2.875318e+03,1.638409e+00,7.260578e+04
min,-1.000000e+00,0.000000e+00,-2.000000e+00,1.100000e+01,-39.000000,0.000000e+00,0.000000e+00,1.748333e+12
25%,1.400000e+01,7.000000e+00,-1.000000e+00,1.100000e+01,-1.000000,3.750000e+03,3.000000e+00,1.748333e+12
50%,3.100000e+01,1.600000e+01,-1.000000e+00,2.100000e+01,0.000000,6.329000e+03,4.000000e+00,1.748333e+12
75%,5.200000e+01,2.800000e+01,-1.000000e+00,3.100000e+01,1.000000,8.343000e+03,4.000000e+00,1.748333e+12
max,1.330000e+02,5.000000e+01,5.000000e+01,3.100000e+01,5.000000,1.035900e+04,5.000000e+00,1.748333e+12


epoch                                               \
                       count       mean        std  min   25%   50%   75%   
strategy                                                                    
CLOSEST_RUN_AWAY    990180.0  41.550534  29.101098 -1.0  17.0  37.0  62.0   
ONLY_ONE_RUN_AWAY   521140.0  22.210700  15.489034 -1.0   9.0  21.0  33.0   
RANDOM_SINGLE      1066316.0  36.412801  25.551537 -1.0  15.0  33.0  53.0   

                           position             ... max_num_snacks       \
                     max      count       mean  ...            75%  max   
strategy                                        ...                       
CLOSEST_RUN_AWAY   133.0   990180.0  21.023136  ...            4.0  5.0   
ONLY_ONE_RUN_AWAY   81.0   521140.0  14.743505  ...            4.0  4.0   
RANDOM_SINGLE      124.0  1066316.0  19.272124  ...            5.0  5.0   

                   timestamp                                            \
                       count          mean           std           min   
strategy                                                                 
CLOSEST_RUN_AWAY    990180.0  1.748333e+12  72328.767313  1.748333e+12   
ONLY_ONE_RUN_AWAY   521140.0  1.748333e+12  62848.715208  1.748333e+12   
RANDOM_SINGLE      1066316.0  1.748333e+12  76792.114401  1.748333e+12   

                                                                           
                            25%           50%           75%           max  
strategy                                                                   
CLOSEST_RUN_AWAY   1.748333e+12  1.748333e+12  1.748333e+12  1.748333e+12  
ONLY_ONE_RUN_AWAY  1.748333e+12  1.748333e+12  1.748333e+12  1.748333e+12  
RANDOM_SINGLE      1.748333e+12  1.748333e+12  1.748333e+12  1.748333e+12  

[3 rows x 64 columns]

In [15]:
ds_by_grouped = []
df_grouped = df.groupby(by="game_id")
for idx, group in df_grouped:
    last_epoch = group.epoch.max()
    df_last_epoch = group[group["epoch"] == last_epoch]
    d = {"end_epoch": last_epoch, "game_id": g_id, "strategy": df_last_epoch["strategy"].iloc[0]}
    for idx, row in df_last_epoch.iterrows():
        d[row["name"]] = GameResult(row["game_result"]).name
    ds_by_grouped.append(d)

In [18]:
df_game_result = pd.DataFrame(ds_by_grouped)
df_game_result["winner"] = df_game_result.apply(get_winner, axis=1)

Exception: Impossible, if any animal finished, at least one animal is not chased

In [ ]:
df_game_result[df_game_result["C"] != GameResult.FINISHED.name]
px.histogram(df_game_result, "winner", color="strategy")


: 

: 

In [ ]:
px.histogram(df_game_result, x="end_epoch", nbins=len(df_game_result.end_epoch.unique()), color="strategy")

: 

: 

: 

: 